In [1]:
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
from ibapi.common import *  # noqa
from threading import Thread
import time
import nest_asyncio
nest_asyncio.apply()

import re
from datetime import datetime, timedelta

In [2]:
class IBApi(EWrapper, EClient):
    def __init__(self):
        EClient.__init__(self, self)
        self.headlines = []  # List to store news headlines
        self.conId = None  # Store the retrieved conId
        self.news_providers = []  # Store available news providers

    def contractDetails(self, reqId, contractDetails):
        self.conId = contractDetails.contract.conId

    def newsProviders(self, newsProviders):
        self.news_providers = [provider.code for provider in newsProviders]

    def historicalNews(self, reqId: int, time: str, providerCode: str, articleId: str, headline: str):
        # Save each headline to the list
        self.headlines.append({
            "time": time,
            "providerCode": providerCode,
            "articleId": articleId,
            "headline": headline
    })

def run_loop(api):
    api.run()

In [3]:
## Initialize the IBApi object
app = IBApi()

# Connect to TWS or IB Gateway
app.connect("127.0.0.1", 7497, clientId=1)

# Start the API thread
api_thread = Thread(target=run_loop, args=(app,), daemon=True)
api_thread.start()

In [4]:
def get_Data(contract, endDate, historicalNewsParam):

    historical_news_format = endDate.strftime("%Y-%m-%d %H:%M:%S") + ".0"

    app.reqContractDetails(1, contract)
    time.sleep(2)  # Wait for the conId to be retrieved
    if app.conId is None:
        raise Exception("Failed to retrieve conId for the contract")

    app.reqNewsProviders()
    time.sleep(2)  # Wait for news providers to be retrieved
    if not app.news_providers:
        raise Exception("Failed to retrieve news providers")

    news_providers = "+".join(app.news_providers)
    #print(news_providers)

    # Request historical news for the contract
    time.sleep(2)  # Wait for connection #Use the startdate, I havent figured how the date range works but we can simply filter out the news later on. :)
    app.reqHistoricalNews(1, app.conId, news_providers, historical_news_format, "", historicalNewsParam['numberOfHeadlines'], None)  # Fetch the latest 10 news items

    # Fetch content for each article
    time.sleep(15) # Wait for news to fetch

    news_headlines = app.headlines
    print(len(news_headlines))
    
    return news_headlines


ERROR -1 2104 Market data farm connection is OK:usfarm.nj
ERROR -1 2104 Market data farm connection is OK:cashfarm
ERROR -1 2104 Market data farm connection is OK:usfarm
ERROR -1 2106 HMDS data farm connection is OK:euhmds
ERROR -1 2106 HMDS data farm connection is OK:fundfarm
ERROR -1 2106 HMDS data farm connection is OK:ushmds
ERROR -1 2158 Sec-def data farm connection is OK:secdefil


In [5]:
contract = Contract()
contract.symbol = "AMD"
contract.secType = "STK"
contract.exchange = "SMART"
contract.currency = "USD"

endDate = datetime.today().replace(hour=0, minute=0, second=0, microsecond=0)
endDate = endDate - timedelta(minutes=15)
#datetime(2023, 5, 8, 21, 44, 0)

In [6]:

historicalNewsParam = {
    'numberOfHeadlines' : 300 #300 being the limit for one call
}

In [13]:
def getAllHeadlines(contract, historicalNewsParam):

    endDate = datetime.today().replace(hour=0, minute=0, second=0, microsecond=0)
    endDate = endDate - timedelta(minutes=15)
    
    final_list = []
    continue_flag = 1
    
    prev_a = 0
    while continue_flag == 1:
        a = get_Data(contract, endDate, historicalNewsParam)
        #print("HAHA", len(a) - prev_a)
        
        if len(a) - prev_a <= 1:
            break
        else:
            final_list += a
            endDate = datetime.strptime(a[-1]['time'], '%Y-%m-%d %H:%M:%S.%f') + timedelta(days=1)
            prev_a = len(a)

    return final_list



In [15]:
headlines = getAllHeadlines(contract, historicalNewsParam)

300
453
454


In [21]:
headlines[-1]

{'time': '2022-02-02 15:02:57.0',
 'providerCode': 'BRFG',
 'articleId': 'BRFG$10d21fb1',
 'headline': '{A:800015:K:1.00:C:1.0:L:en}Advanced Micro navigates a supply constrained environment to deliver excellent Q4 results'}

In [25]:
app.disconnect()